In [65]:
from importlib import reload
import os
from toolz import pipe as p

import scrapDataHelpers

In [53]:
ls image_data/

gossiping/  laughing/     pullinghair/  quarrel/   stabbing/
isolation/  nonbullying/  punching/     slapping/  strangle/


In [58]:
len([f for d in os.listdir('image_data') for 
     f in p(os.path.join('image_data', d), os.listdir)])

n_train = 1500
n_val = 400

dest_dir = 'scrap_data' + str(n_train)

In [66]:
reload(scrapDataHelpers)

<module 'scrapDataHelpers' from '/home/john/projects/dl/scrapDataHelpers.py'>

In [56]:
classes = os.listdir('image_data')
scrapDataHelpers.makeScrapData(classes, n_train=n_train, n_val=n_val)

In [76]:
scrapDataHelpers.classPercentages(dest_dir)

{('train', 'strangle'): 14.333333333333334,
 ('val', 'strangle'): 15.0,
 ('train', 'quarrel'): 9.533333333333333,
 ('val', 'quarrel'): 10.75,
 ('train', 'gossiping'): 16.2,
 ('val', 'gossiping'): 16.25,
 ('train', 'pullinghair'): 10.733333333333334,
 ('val', 'pullinghair'): 10.5,
 ('train', 'nonbullying'): 3.8,
 ('val', 'nonbullying'): 5.25,
 ('train', 'isolation'): 10.2,
 ('val', 'isolation'): 8.5,
 ('train', 'slapping'): 7.866666666666666,
 ('val', 'slapping'): 9.5,
 ('train', 'laughing'): 7.066666666666667,
 ('val', 'laughing'): 3.5000000000000004,
 ('train', 'punching'): 14.466666666666667,
 ('val', 'punching'): 15.25,
 ('train', 'stabbing'): 5.800000000000001,
 ('val', 'stabbing'): 5.5}

# Just try 2 classes

In [78]:
scrapDataHelpers.makeScrapData(['laughing', 'punching'], n_train=50, n_val=10)

In [79]:
scrapDataHelpers.classPercentages('scrap_data50/')

{('train', 'laughing'): 42.0,
 ('val', 'laughing'): 40.0,
 ('train', 'punching'): 57.99999999999999,
 ('val', 'punching'): 60.0}